<a href="https://colab.research.google.com/github/BatoolMM/DeepSulfation/blob/master/Pre_process_and_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils

In [2]:
import pandas as pd
import csv
import numpy as np
import keras.utils.np_utils as kutils
import matplotlib.pyplot as plt
from utils import *
import fastbook
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner

In [14]:
file = '/content/CG-20-306_SD1.xlsx'
training = pd.read_excel(file, sheet_name=0)
test =pd.read_excel(file, sheet_name=1)

In [15]:
training.head(2)

,POSITIVE,Unnamed: 1,Unnamed: 2
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,2.0
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,2.0


In [16]:
removed_row = training['POSITIVE'].eq('NEGATIVE')
mask = removed_row.cumsum().eq(0)
training['Target'] = np.where(mask, '1', '0')
training = training[~removed_row].copy()
training.head(2)


,POSITIVE,Unnamed: 1,Unnamed: 2,Target
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,2.0,1
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,2.0,1


In [17]:
training.columns

Index(['POSITIVE', 'Unnamed: 1', 'Unnamed: 2', 'Target'], dtype='object')

In [18]:
training.rename(
    columns = {'POSITIVE':'ID',
               'Unnamed: 1':'seq',
               'Unnamed: 2': 'site',
               'Target': 'Label'}, 
               inplace = True)
training.drop('site', axis=1, inplace=True)

def indexe(string, char):
  for i, j in enumerate(char):
    if j == char :
      yield i

def ind(my_list, desired_element):
    return [index for index, element in enumerate(my_list) if element == desired_element]


training['pos'] = training['seq'].apply(lambda x: ind(x,'Y'))
training.head(5)

,ID,seq,Label,pos
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,1,"[20, 27]"
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,1,"[13, 20, 39]"
2,P10493,TAKGVVSADVNLDLDDDGADYEDEDYDLVTSHLGLEDVATP,1,"[20, 25]"
3,P10493,VSADVNLDLDDDGADYEDEDYDLVTSHLGLEDVATPSPSHS,1,"[15, 20]"
4,P13521,ENKPYALNSEKNFPMDMSDDYETQQWPERKLKHMQFPPMYE,1,"[4, 20, 39]"


In [19]:
test.tail(2)

,POSITIVE,Unnamed: 1,Unnamed: 2
159,P08238,KNIVKKCLELFSELAEDKENYKKFYEAFSKNLKLGIHEDST,NaN
160,Q03834,MLHSDLATFCLAIREEPGNFYNETQLDSSTIVQKLNTKIFG,NaN


In [23]:
removed_row_test = test['POSITIVE'].eq('NEGATIVE')
mask = removed_row_test.cumsum().eq(0)
test['Target'] = np.where(mask, '1', '0')
test = test[~removed_row_test].copy()
test.head(5)

,POSITIVE,Unnamed: 1,Unnamed: 2,Target
0,P49682,DHQVLNDAEVAALLENFSSSYDYGENESDSCCTSPPCPQDF,6.0,1
1,P49682,QVLNDAEVAALLENFSSSYDYGENESDSCCTSPPCPQDFSL,6.0,1
2,Q7M3V5,DLMDFLLEYEDEDRSKRFDDYGHMRFGKRGGEEQFDDYGHM,2.0,1
3,Q8NBP7,LLLLLLGPAGARAQEDEDGDYEELVLALRSEEDGLAEAPEH,23.0,1
4,Q9NZ53,PGSGFPSEENEESRILQPPQYFWEEEEELNDSSLDLGPTAD,7.0,1


In [29]:
test.rename(
    columns = {'POSITIVE':'ID',
               'Unnamed: 1':'seq',
               'Unnamed: 2': 'site',
               'Target': 'Label'},
            inplace = True)
test.drop('site', axis=1, inplace=True)

In [33]:
#test['pos'] = test['seq'].apply(lambda x: for i, j in enumerate(x) if j = 'Y' )
#test['pos'] = test['seq'].apply(lambda x: ind(x,'Y'))

test['pos'] = test['seq'].apply(lambda x: ind(x,'Y'))

test.head(5)

,ID,seq,Label,pos
0,P49682,DHQVLNDAEVAALLENFSSSYDYGENESDSCCTSPPCPQDF,1,"[20, 22]"
1,P49682,QVLNDAEVAALLENFSSSYDYGENESDSCCTSPPCPQDFSL,1,"[18, 20]"
2,Q7M3V5,DLMDFLLEYEDEDRSKRFDDYGHMRFGKRGGEEQFDDYGHM,1,"[8, 20, 37]"
3,Q8NBP7,LLLLLLGPAGARAQEDEDGDYEELVLALRSEEDGLAEAPEH,1,[20]
4,Q9NZ53,PGSGFPSEENEESRILQPPQYFWEEEEELNDSSLDLGPTAD,1,[20]


In [34]:
training.to_csv('cleaned_training.csv')
test.to_csv('cleaned_test.csv')

In [35]:
def getMatrixLabel(positive_position_file_name,sites, window_size=51, empty_aa = '*'):
    # input format   label, proteinName, postion,shortsequence,
    prot = []  # list of protein name
    pos = []  # list of position with protein name
    rawseq = []
    all_label = []

    short_seqs = []
    half_len = (window_size - 1) / 2

    with open(positive_position_file_name, 'r') as rf:
        reader = csv.reader(rf)
        for row in reader:

            position = int(row[2])
            sseq = row[3]
            rawseq.append(row[3])
            center = sseq[position - 1]
            if center in sites:
                all_label.append(int(row[0]))
                prot.append(row[1])
                pos.append(row[2])

                #short seq
                if position - half_len > 0:
                    start = position - half_len
                    left_seq = sseq[start - 1:position - 1]
                else:
                    left_seq = sseq[0:position - 1]

                end = len(sseq)
                if position + half_len < end:
                    end = position + half_len
                right_seq = sseq[position:end]

                if len(left_seq) < half_len:
                    nb_lack = half_len - len(left_seq)
                    left_seq = ''.join([empty_aa for count in range(nb_lack)]) + left_seq

                if len(right_seq) < half_len:
                    nb_lack = half_len - len(right_seq)
                    right_seq = right_seq + ''.join([empty_aa for count in range(nb_lack)])
                shortseq = left_seq + center + right_seq
                short_seqs.append(shortseq)
                # coding = one_hot_concat(shortseq)
                # all_codings.append(coding)

        targetY = kutils.to_categorical(all_label)

        ONE_HOT_SIZE = 21
        # _aminos = 'ACDEFGHIKLMNPQRSTVWY*'
        letterDict = {}
        letterDict["A"] = 0
        letterDict["C"] = 1
        letterDict["D"] = 2
        letterDict["E"] = 3
        letterDict["F"] = 4
        letterDict["G"] = 5
        letterDict["H"] = 6
        letterDict["I"] = 7
        letterDict["K"] = 8
        letterDict["L"] = 9
        letterDict["M"] = 10
        letterDict["N"] = 11
        letterDict["P"] = 12
        letterDict["Q"] = 13
        letterDict["R"] = 14
        letterDict["S"] = 15
        letterDict["T"] = 16
        letterDict["V"] = 17
        letterDict["W"] = 18
        letterDict["Y"] = 19
        letterDict["*"] = 20

        Matr = np.zeros((len(short_seqs), window_size, ONE_HOT_SIZE))
        samplenumber = 0
        for seq in short_seqs:
            AANo = 0
            for AA in seq:
                index = letterDict[AA]
                Matr[samplenumber][AANo][index] = 1
                AANo = AANo+1
            samplenumber = samplenumber + 1

    return Matr, targetY


In [22]:
#minimal example:
d = {'col1': ['SYDYGENESD', 'SSYDYGENE'], 
'col2': ['A','B' ]}

data = pd.DataFrame(data=d)
data.head()

data['new_col'] = data['col1'].str.index('Y')
data.head()

target = {'col1': ['SYDYGENESD', 'SSYDYGENE'],
          'col2': ['A','B'],
          'col3':['1, 3','2, 4']}
target = pd.DataFrame(data=target)
target.head()

,col1,col2,col3
0,SYDYGENESD,A,"1, 3"
1,SSYDYGENE,B,"2, 4"
